# **⚖️ EMPLOYMENT LAW: CASE LAW DISCOVERY SYATEM UNDER KENYA LAW**

## **PROJECT OBJECTIVE**

* ### To build a smart legal research tool that helps advocates quickly find accurate, relevant, and authoritative employment law cases from Kenya Law based on legal issues described in plain English.

## **SCOPE**

### Area:

* ### Employment & Labour Relations

### Courts:

* ### Employment and Labour Relations Court (ELRC)

* ### Court of Appeal (employment cases)

* ### Supreme Court (where applicable)

### Jurisdiction:

* ### Kenya

### Output:

* ### Research assistance only (NOT legal advice)

## **METHODOLOGY**

STEP 1: DATA COLLECTION (KENYA LAW)

STEP 2: LEGAL TEXT PREPROCESSING

STEP 3: EMBEDDINGS (CORE INTELLIGENCE)

STEP 4: SEMANTIC SEARCH (THE MAGIC)

STEP 5: RELEVANCE & AUTHORITY RANKING

STEP 6: DISPLAY MEANINGFUL RESULTS

STEP 7: USER INTERFACE (STREAMLIT)

STEP 8: LEGAL & ETHICAL SAFEGUARDS

# **DATA COLLECTION FROM KENYA LAW**

## STEP 1: MANUALLY COLLECT JUDGMENT LINKS

In [3]:
# Importing libraries

import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import time
import re
import json

import torch
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

C:\Users\BEST\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
base_url = "https://new.kenyalaw.org/judgments/KEELRC/"

In [5]:
headers = {
    "User-Agent": "EmploymentLawResearchBot/1.0 (Academic Project)"
}

response = requests.get(base_url, headers=headers, timeout=20)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")

## STEP 2: EXTRACT CORE METADATA

In [6]:
# Case title
title = soup.find("h1")
case_title = title.get_text(strip=True) if title else "Unknown Title"

# Court
court_tag = soup.find("meta", {"name": "court"})
court = court_tag["content"] if court_tag else "ELRC"

# Date
date_tag = soup.find("meta", {"name": "date"})
decision_date = date_tag["content"] if date_tag else "Unknown Date"

print(case_title, court, decision_date)

Employment and Labour Relations Court ELRC Unknown Date


## STEP 3: EXTRACT THE JUDGMENT BODY

In [7]:
for tag in soup.find_all(["article", "section", "div"]):
    text = tag.get_text(strip=True)
    if len(text) > 1000:
        print(tag.name, tag.get("class"))


div ['container', 'pb-5']
div ['row']
div ['col-lg-3', 'd-none', 'd-lg-block', 'document-list-facets-wrapper', 'pocketlaw-hidden']
div ['col']
div ['position-relative']
div ['table-responsive']


In [8]:
judgment_div = soup.find(
    lambda tag: tag.name in ["div", "article", "section"]
    and tag.get_text(strip=True)
    and len(tag.get_text(strip=True)) > 2000
)

if not judgment_div:
    raise ValueError("Judgment body not found")

judgment_text = judgment_div.get_text(separator="\n", strip=True)


In [9]:
print("===== JUDGMENT PREVIEW =====")
print(judgment_text[:700])
print("\n===== END PREVIEW =====")


===== JUDGMENT PREVIEW =====
Employment and Labour Relations Court
31,173
  
    
      judgments
Advanced search
Court stations
Employment and Labour Relations at Garissa
Employment and Labour Relations Court at Bungoma
Employment and Labour Relations Court at Eldoret
Employment and Labour Relations Court at Kakamega
Employment and Labour Relations Court at Kericho
Employment and Labour Relations Court at Kisii
Employment and Labour Relations Court at Kisumu
Employment and Labour Relations Court at Kitale
Employment and Labour Relations Court at Machakos
Employment and Labour Relations Court at Malindi
Employment and Labour Relations Court at Meru
Employment and Labour Relations Court at Mombasa
Employment and Labour R

===== END PREVIEW =====


# **CLEAN LEGAL TEXT TO IMPROVE QUALITY**

In [10]:
def clean_akn_text(text):
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

clean_text = clean_akn_text(judgment_text)

# **SAVE STRUCTURED OUTPUT**

In [11]:
os.makedirs("data/akn_cases", exist_ok=True)

case_data = {
    "case_title": case_title,
    "court": court,
    "decision_date": decision_date,
    "source_url": base_url,
    "text": clean_text
}

file_name = "keelrc_2025_962.json"

with open(f"data/akn_cases/{file_name}", "w", encoding="utf-8") as f:
    json.dump(case_data, f, ensure_ascii=False, indent=2)

# **SPLIT INTO SEARCHABLE CHUNKS (FOR NLP)**

In [12]:
def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks

chunks = chunk_text(clean_text)

# **CREATE EMBEDDINGS**

In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)


In [14]:
print(embeddings.shape)

(4, 384)


In [24]:
print(embeddings)

[[-0.06183275  0.05020792 -0.03339604 ... -0.0477563  -0.01317014
   0.00969268]
 [-0.07945757  0.01062756  0.04711544 ... -0.10500664 -0.04604482
   0.02960407]
 [-0.0929507   0.07297579  0.01184011 ... -0.13978249  0.05527228
   0.04620954]
 [-0.0987413   0.05076386 -0.00768123 ... -0.15003024  0.02399292
   0.03449104]]


# **CREATE CHROMADB CLIENT - PERSISTENT**

In [25]:
import chromadb
import numpy as np

In [28]:
# 1. Setup Persistent Storage for the Kenyan Law DB
db_path = "./kenya_law_db"
client = chromadb.PersistentClient(path=db_path)

In [29]:
# 2. Use 'cosine' similarity for better semantic matching in legal texts
collection = client.get_or_create_collection(
    name="employment_case_law",
    metadata={"hnsw:space": "cosine"} 
)

In [30]:
# 3. Prepare Data for "Wide" Discovery
# We map each chunk to the original case metadata so lawyers know which case the answer came from.

ids = []
metadatas = []
documents = []

for i, chunk in enumerate(chunks):
    ids.append(f"{file_name}_chunk_{i}")
    documents.append(chunk)
    metadatas.append({
        "case_title": case_data["case_title"],
        "court": case_data["court"],
        "decision_date": case_data["decision_date"],
        "source_url": case_data["source_url"],
        "chunk_index": i
    })

In [31]:
# 4. Add to ChromaDB
# embeddings was already generated by your model.encode(chunks)
collection.add(
    embeddings=embeddings.tolist(), # Convert numpy array to list
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print(f"Successfully indexed {len(chunks)} searchable segments for: {case_title}")

Successfully indexed 4 searchable segments for: Employment and Labour Relations Court


In [32]:
# Example Search: A lawyer looking for "Summary Dismissal procedure"
query_text = "What is the procedure for summary dismissal in Kenya?"
query_embedding = model.encode([query_text]).tolist()

results = collection.query(
    query_embeddings=query_embedding,
    n_results=3,
    include=["documents", "metadatas", "distances"]
)

for i in range(len(results['documents'][0])):
    print(f"\n--- Result {i+1} (Score: {results['distances'][0][i]:.4f}) ---")
    print(f"CASE: {results['metadatas'][0][i]['case_title']}")
    print(f"TEXT: {results['documents'][0][i][:300]}...")


--- Result 1 (Score: 0.4477) ---
CASE: Employment and Labour Relations Court
TEXT: Employment and Labour Relations Court 31,173 judgments Advanced search Court stations Employment and Labour Relations at Garissa Employment and Labour Relations Court at Bungoma Employment and Labour Relations Court at Eldoret Employment and Labour Relations Court at Kakamega Employment and Labour R...

--- Result 2 (Score: 0.4567) ---
CASE: Employment and Labour Relations Court
TEXT: Service Board Uasin Gishu County & 2 others (Petition E010 of 2025) [2025] KEELRC 3720 (KLR) (22 December 2025) (Ruling) 22 December 2025 Odoyo v Universities Academic Staff Union (UASU); Registrar of Trade Unions (Interested Party) (Petition E013 of 2025) [2025] KEELRC 3746 (KLR) (22 December 2025)...

--- Result 3 (Score: 0.5143) ---
CASE: Employment and Labour Relations Court
TEXT: 3678 (KLR) (18 December 2025) (Judgment) 18 December 2025 Apex Steel Limited v Mauti (Employment and Labour Relations Appeal 3 of 2023) [202

In [40]:
def generate_citation_aware_answer(query, collection, model):
    # 1. Search the Vector DB
    query_embedding = model.encode([query]).tolist()
    
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3,
        include=["documents", "metadatas"]
    )

    # 2. Build the Context String with Citations
    context_parts = []
    for i in range(len(results['documents'][0])):
        doc = results['documents'][0][i]
        meta = results['metadatas'][0][i]
        source_label = f"[{meta['case_title']} ({meta['court']}, {meta['decision_date']})]"
        context_parts.append(f"Source {source_label}:\n{doc}")

    full_context = "\n\n".join(context_parts)

    # 3. Create the Prompt (MUST be inside the function)
    prompt = f"""
    You are a Kenyan Employment Law Assistant. Use the following legal fragments to answer the user's question. 
    
    STRICT RULES:
    1. Only use the provided context. 
    2. You MUST cite the case title and court for every claim you make.
    3. If the answer is not in the context, say you don't know based on the current records.

    CONTEXT:
    {full_context}

    QUESTION: 
    {query}

    ANSWER:
    """
    
    return prompt, results['metadatas'][0]

In [41]:
# Test it with a real legal question
my_question = "What are the grounds for summary dismissal?"

# This calls the function and saves the output to 'final_prompt'
final_prompt, metadata_list = generate_citation_aware_answer(my_question, collection, model)

print(final_prompt)


    You are a Kenyan Employment Law Assistant. Use the following legal fragments to answer the user's question. 

    STRICT RULES:
    1. Only use the provided context. 
    2. You MUST cite the case title and court for every claim you make.
    3. If the answer is not in the context, say you don't know based on the current records.

    CONTEXT:
    Source [Employment and Labour Relations Court (ELRC, Unknown Date)]:
KEELRC 3730 (KLR) (19 December 2025) (Judgment) 19 December 2025 SI. Mwaura & Company Advocates v Meditest Diagnostics Services Limited (Miscellaneous Application E352 of 2024) [2025] KEELRC 3704 (KLR) (19 December 2025) (Ruling) 19 December 2025 Wambugu v County Government of Nyandarua & 2 others (Cause E024 of 2025) [2025] KEELRC 3729 (KLR) (19 December 2025) (Judgment) 19 December 2025 Wambui v Wagika Holdings Limited; Kariuki & 2 others (Objector) (Cause 472 of 2017) [2025] KEELRC 3697 (KLR) (19 December 2025) (Ruling) 19 December 2025 Wangai v Egerton University (C